In [10]:
import os, sys
from pathlib import Path

# 1) Check which Python your notebook is using (should be inside .venv)
print("Python executable:", sys.executable)

# 2) Point the notebook at your project folder explicitly
PROJECT_DIR = Path("/Users/adyasamishra/python-playground/Vendor analysis").resolve()
os.chdir(PROJECT_DIR)
print("Now in:", Path.cwd())

# 3) Verify the data folder and list CSVs
DATA_DIR = PROJECT_DIR / "data"
print("DATA_DIR exists:", DATA_DIR.exists())
print("CSV files:", sorted(p.name for p in DATA_DIR.glob("*.csv")))


Python executable: /Users/adyasamishra/python-playground/Vendor analysis/.venv/bin/python
Now in: /Users/adyasamishra/python-playground/Vendor analysis
DATA_DIR exists: True
CSV files: ['begin_inventory.csv', 'end_inventory.csv', 'purchase_prices.csv', 'purchases.csv', 'sales.csv', 'vendor_invoice.csv']


In [25]:
# If you see "ModuleNotFoundError", run this once:
# %pip install pandas sqlalchemy --quiet

import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
import logging


In [27]:
import logging, sys
from pathlib import Path

# make sure the folder exists
Path("logs").mkdir(exist_ok=True)

# simple config: write to logs/run.log + also show in the notebook output
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler("logs/run.log", encoding="utf-8"),
        logging.StreamHandler(sys.stdout),
    ],
)

logging.info("Logger ready")


2025-09-26 19:09:14,809 | INFO | Logger ready


In [28]:
import pandas as pd
from pathlib import Path
import logging

DATA_DIR = Path("data")

logging.info("Starting CSV ingest")

for f in sorted(DATA_DIR.glob("*.csv")):
    try:
        logging.info(f"Reading {f.name}")
        df = pd.read_csv(f)
        logging.info(f"{f.name} shape={df.shape}")
        df.to_sql(f.stem, engine, if_exists="replace", index=False)
        logging.info(f"Wrote table '{f.stem}' rows={len(df)}")
    except Exception:
        logging.exception(f"FAILED on {f.name}")

logging.info("CSV ingest completed")


2025-09-26 19:10:47,505 | INFO | Starting CSV ingest
2025-09-26 19:10:47,515 | INFO | Reading begin_inventory.csv
2025-09-26 19:10:47,817 | INFO | begin_inventory.csv shape=(206529, 9)
2025-09-26 19:10:49,608 | INFO | Wrote table 'begin_inventory' rows=206529
2025-09-26 19:10:49,610 | INFO | Reading end_inventory.csv
2025-09-26 19:10:49,830 | INFO | end_inventory.csv shape=(224489, 9)
2025-09-26 19:10:50,740 | INFO | Wrote table 'end_inventory' rows=224489
2025-09-26 19:10:50,741 | INFO | Reading purchase_prices.csv
2025-09-26 19:10:50,758 | INFO | purchase_prices.csv shape=(12261, 9)
2025-09-26 19:10:50,814 | INFO | Wrote table 'purchase_prices' rows=12261
2025-09-26 19:10:50,814 | INFO | Reading purchases.csv
2025-09-26 19:10:53,976 | INFO | purchases.csv shape=(2372474, 16)
2025-09-26 19:11:22,404 | INFO | Wrote table 'purchases' rows=2372474
2025-09-26 19:11:22,442 | INFO | Reading sales.csv
2025-09-26 19:11:35,198 | INFO | sales.csv shape=(12825363, 14)
2025-09-26 19:26:17,599 | I

In [17]:
# 1) Create/open the SQLite database file in your project folder
engine = create_engine('sqlite:///inventory.db')

In [18]:
# 2) Point to the data folder
DATA_DIR = Path("data")
print("Found CSVs:")
for f in sorted(DATA_DIR.glob("*.csv")):
    print(" -", f.name)

Found CSVs:
 - begin_inventory.csv
 - end_inventory.csv
 - purchase_prices.csv
 - purchases.csv
 - sales.csv
 - vendor_invoice.csv


In [20]:
# 3) Helper to write a DataFrame as a SQL table
def ingest_db(df: pd.DataFrame, table_name: str, engine):
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)
    print(f" → wrote table '{table_name}' with {len(df):,} rows")

In [21]:
# 4) Loop through all CSVs and ingest each as a table (filename -> table name)
for f in sorted(DATA_DIR.glob("*.csv")):
    df = pd.read_csv(f)
    print(f.name, df.shape)
    ingest_db(df, f.stem, engine)   # f.stem is filename without .csv


begin_inventory.csv (206529, 9)
 → wrote table 'begin_inventory' with 206,529 rows
end_inventory.csv (224489, 9)
 → wrote table 'end_inventory' with 224,489 rows
purchase_prices.csv (12261, 9)
 → wrote table 'purchase_prices' with 12,261 rows
purchases.csv (2372474, 16)
 → wrote table 'purchases' with 2,372,474 rows
sales.csv (12825363, 14)
 → wrote table 'sales' with 12,825,363 rows
vendor_invoice.csv (5543, 10)
 → wrote table 'vendor_invoice' with 5,543 rows


In [22]:
import pandas as pd

tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%' ORDER BY name;",
    engine
)
tables


,name
0,begin_inventory
1,end_inventory
2,purchase_prices
3,purchases
4,sales
5,vendor_invoice


In [24]:
import pandas as pd
pd.read_sql("PRAGMA table_info(purchases);", engine)


,cid,name,type,notnull,dflt_value,pk
0,0,InventoryId,TEXT,0,None,0
1,1,Store,BIGINT,0,None,0
2,2,Brand,BIGINT,0,None,0
3,3,Description,TEXT,0,None,0
4,4,Size,TEXT,0,None,0
5,5,VendorNumber,BIGINT,0,None,0
6,6,VendorName,TEXT,0,None,0
7,7,PONumber,BIGINT,0,None,0
8,8,PODate,TEXT,0,None,0
9,9,ReceivingDate,TEXT,0,None,0


In [30]:
import os, time, logging
import pandas as pd
from sqlalchemy import create_engine

# make sure logs/ exists
os.makedirs("logs", exist_ok=True)

# simple file logger, same as video
logging.basicConfig(
    filename="logs/ingestion_db.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a",
    force=True,   # important in notebooks so it resets any previous logging config
)

logging.info("Logging configured")


In [31]:
engine = create_engine("sqlite:///inventory.db")


In [32]:
def ingest_db(df: pd.DataFrame, table_name: str, engine):
    """Write a DataFrame into SQLite as `table_name` (replace if exists)."""
    df.to_sql(table_name, con=engine, if_exists="replace", index=False)

def load_raw_data(data_dir: str = "data"):
    """Load all CSVs from `data_dir` into the DB, logging each file and total time."""
    start = time.time()
    for file in os.listdir(data_dir):
        if file.endswith(".csv"):
            try:
                logging.info("Ingesting %s in db", file)
                df = pd.read_csv(os.path.join(data_dir, file))
                print(df.shape)  # the video prints the shape to the cell output
                ingest_db(df, file[:-4], engine)  # strip ".csv" to use as table name
                logging.info("Completed %s | rows=%d", file, len(df))
            except Exception:
                logging.exception("FAILED on %s", file)
    total_min = (time.time() - start) / 60
    logging.info("========= Ingestion Complete =========")
    logging.info("Total Time Taken: %.2f minutes", total_min)


In [33]:
load_raw_data("data")


(2372474, 16)
(12261, 9)
(5543, 10)
(206529, 9)
(224489, 9)
(12825363, 14)


In [34]:
import pandas as pd
import logging
from pathlib import Path

# KPI: Top spend by vendor
logging.info("KPI: Top spend by vendor — start")

sql = """
SELECT
  "VendorNumber" AS vendor_id,
  "VendorName"   AS vendor_name,
  SUM("Dollars") AS spend
FROM purchases
GROUP BY "VendorNumber","VendorName"
ORDER BY spend DESC
LIMIT 20;
"""

spend_by_vendor = pd.read_sql(sql, engine)
logging.info("KPI: Top spend by vendor — done | rows=%d", len(spend_by_vendor))

# Save for your dashboard later
Path("outputs").mkdir(exist_ok=True)
spend_by_vendor.to_csv("outputs/spend_by_vendor.csv", index=False)
logging.info("Saved outputs/spend_by_vendor.csv")

spend_by_vendor  # show in the notebook


,vendor_id,vendor_name,spend
0,3960,DIAGEO NORTH AMERICA INC,50959796.85
1,4425,MARTIGNETTI COMPANIES,27821473.91
2,12546,JIM BEAM BRANDS COMPANY,24203151.05
3,17035,PERNOD RICARD USA,24124091.56
4,480,BACARDI USA INC,17624378.72
5,1392,CONSTELLATION BRANDS INC,15573917.90
6,1128,BROWN-FORMAN CORP,13529433.08
7,9165,ULTRA BEVERAGE COMPANY LLP,13210613.93
8,3252,E & J GALLO WINERY,12289608.09
9,9552,M S WALKER INC,10935817.30
